In [1]:
import os
import numpy as np
import torch
from PIL import Image
import yaml


class Dataset(object):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "photo"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "mask"))))
        self.info = list(sorted(os.listdir(os.path.join(root, "info"))))

    def __getitem__(self, idx):
        # load images and masks
        img_path = os.path.join(self.root, "photo", self.imgs[idx])
        mask_path = os.path.join(self.root, "mask", self.masks[idx])
        info_path = os.path.join(self.root, "info", self.info[idx])
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path)
        # convert the PIL Image into a numpy array
        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)

        # first id is the background, so remove it
        obj_ids = obj_ids[1:]
        
        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
       
        label_names = []
        with open(info_path) as f:
            temp = yaml.load(f.read())
            label_names = temp['label_names']
            del label_names[0]
        labels = []
#         if(num_objs!=len(label_names)):
#         print(num_objs, label_names)
        for i in range(len(label_names)):
            if(label_names[i].find("a")!=-1):
                labels.append(1)
            elif(label_names[i].find("b")!=-1):
                labels.append(2)
       
        labels = torch.tensor(labels) 
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target, image_id

    def __len__(self):
        return len(self.imgs)
    
    

In [2]:
import transforms as T
import utils

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [3]:
from engine import train_one_epoch, evaluate
import utils


# train on the GPU or on the CPU, if a GPU is not available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


# use our dataset and defined transformations
dataset = Dataset('train_data', get_transform(train=True))
dataset_test = Dataset('eval_data', get_transform(train=False))

In [4]:
# split the dataset in train and test set
# indices = torch.randperm(len(dataset)).tolist()
# dataset = torch.utils.data.Subset(dataset, indices[:-50])
# dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, 
    collate_fn=utils.collate_fn)

In [5]:
# import torch
# import torchvision
# from torchvision.models.detection import MaskRCNN
# from torchvision.models.detection.anchor_utils import AnchorGenerator

# backbone = torchvision.models.mobilenet_v2(pretrained=True).features
# backbone.out_channels = 1280
# anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),aspect_ratios=((0.5, 1.0, 2.0),))

# roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'], output_size=7, sampling_ratio=2)
# mask_roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'], output_size=14, sampling_ratio=2)

# # put the pieces together inside a MaskRCNN model
# model = MaskRCNN(backbone, num_classes=3, rpn_anchor_generator=anchor_generator, box_roi_pool=roi_pooler, mask_roi_pool=mask_roi_pooler)

In [6]:
# import torch
# import torchvision
# from torchvision.models.detection import RetinaNet
# from torchvision.models.detection.anchor_utils import AnchorGenerator

# def get_model_instance_segmentation(num_classes):
#      # load a pre-trained model for classification and return
#      # only the features
#     backbone = torchvision.models.mobilenet_v2(pretrained=True).features
#      # RetinaNet needs to know the number of
#     # output channels in a backbone. For mobilenet_v2, it's 1280
#      # so we need to add it here
#     backbone.out_channels = 1280
#     # let's make the network generate 5 x 3 anchors per spatial
#     # location, with 5 different sizes and 3 different aspect
#     # ratios. We have a Tuple[Tuple[int]] because each feature
#     # map could potentially have different sizes and
#     # aspect ratios
#     anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),), aspect_ratios=((0.5, 1.0, 2.0),))

#     # put the pieces together inside a RetinaNet model
#     model = RetinaNet(backbone, num_classes, anchor_generator=anchor_generator)
#     return model

In [10]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=True)

#     # get number of input features for the classifier
#     in_features = model.roi_heads.box_predictor.cls_score.in_features
#     # replace the pre-trained head with a new one
#     model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

#     # now get the number of input features for the mask classifier
#     in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
#     hidden_layer = 256
#     # and replace the mask predictor with a new one
#     model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
#                                                        hidden_layer,
#                                                        num_classes)

    return model

In [11]:
# get the model using our helper function
# model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True, progress=True, 
#                                                            num_classes=2)
model = get_model_instance_segmentation(3) #background, a, b
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005, nesterov=True)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [12]:
import datetime
# let's train it for 10 epochs
num_epochs = 100
print(datetime.datetime.now())
for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

print("That's it!")
print(datetime.datetime.now())

2021-07-15 23:27:57.060079


c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:57: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\torch\nn\functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum'

Epoch: [0]  [  0/343]  eta: 0:08:43  lr: 0.000020  loss: 1.8068 (1.8068)  classification: 1.6372 (1.6372)  bbox_regression: 0.1696 (0.1696)  time: 1.5260  data: 0.1330  max mem: 2960
Epoch: [0]  [ 10/343]  eta: 0:03:57  lr: 0.000166  loss: 1.7075 (1.4596)  classification: 1.6039 (1.3076)  bbox_regression: 0.1696 (0.1520)  time: 0.7128  data: 0.2181  max mem: 4239
Epoch: [0]  [ 20/343]  eta: 0:03:35  lr: 0.000312  loss: 0.6230 (0.9508)  classification: 0.4672 (0.8050)  bbox_regression: 0.1359 (0.1457)  time: 0.6243  data: 0.2231  max mem: 4251
Epoch: [0]  [ 30/343]  eta: 0:03:15  lr: 0.000458  loss: 0.2799 (0.7450)  classification: 0.1859 (0.6110)  bbox_regression: 0.1064 (0.1340)  time: 0.5776  data: 0.1979  max mem: 4251
Epoch: [0]  [ 40/343]  eta: 0:03:04  lr: 0.000604  loss: 0.2368 (0.6173)  classification: 0.1229 (0.4914)  bbox_regression: 0.0973 (0.1259)  time: 0.5489  data: 0.1865  max mem: 4251
Epoch: [0]  [ 50/343]  eta: 0:02:56  lr: 0.000750  loss: 0.2087 (0.5323)  classificat

  2%|▏         | 2/96 [00:00<00:08, 11.56it/s]

Epoch: [0]  [342/343]  eta: 0:00:00  lr: 0.005000  loss: 0.1369 (0.1960)  classification: 0.0635 (0.1141)  bbox_regression: 0.0734 (0.0819)  time: 0.5850  data: 0.2068  max mem: 4254
Epoch: [0] Total time: 0:03:21 (0.5885 s / it)


...

creating index...
index created!
Test:  [ 0/96]  eta: 0:00:21  model_time: 0.1220 (0.1220)  evaluator_time: 0.0020 (0.0020)  time: 0.2230  data: 0.0940  max mem: 4281
Test:  [95/96]  eta: 0:00:00  model_time: 0.1090 (0.1105)  evaluator_time: 0.0010 (0.0014)  time: 0.1808  data: 0.0651  max mem: 4281
Test: Total time: 0:00:17 (0.1860 s / it)
Averaged stats: model_time: 0.1090 (0.1105)  evaluator_time: 0.0010 (0.0014)
Accumulating evaluation results...
DONE (t=0.02s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.983
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.994
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.993
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.983
 Average Recall     (AR) @[ Io

  2%|▏         | 2/96 [00:00<00:07, 12.58it/s]

Epoch: [99]  [342/343]  eta: 0:00:00  lr: 0.000000  loss: 0.0424 (0.0536)  classification: 0.0039 (0.0112)  bbox_regression: 0.0395 (0.0424)  time: 0.5891  data: 0.2026  max mem: 4281
Epoch: [99] Total time: 0:03:20 (0.5851 s / it)


100%|██████████| 96/96 [00:07<00:00, 13.45it/s]


creating index...
index created!
Test:  [ 0/96]  eta: 0:00:22  model_time: 0.1310 (0.1310)  evaluator_time: 0.0020 (0.0020)  time: 0.2340  data: 0.0960  max mem: 4281
Test:  [95/96]  eta: 0:00:00  model_time: 0.1090 (0.1104)  evaluator_time: 0.0010 (0.0014)  time: 0.1821  data: 0.0650  max mem: 4281
Test: Total time: 0:00:17 (0.1864 s / it)
Averaged stats: model_time: 0.1090 (0.1104)  evaluator_time: 0.0010 (0.0014)
Accumulating evaluation results...
DONE (t=0.02s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.983
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.994
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.993
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.983
 Average Recall     (AR) @[ Io

In [13]:

now = datetime.datetime.now()
print(now)
now_str=now.strftime("%Y%m%d")
path = 'model/'+now_str+'retinaNet+fpn.pth'

torch.save(model,path)

2021-07-16 05:43:40.508698


In [ ]:
import cv2
from src.utils import draw_segmentation_map, get_outputs
# from src.coco_names import COCO_INSTANCE_CATEGORY_NAMES as coco_names
from torchvision import transforms

def single_img_predict(path, img, score, model):
    path = os.path.join(path, img)
#     image = np.array(Image.open(path))
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image = cv2.imread(path)
    orig_image = image.copy()
    transform = transforms.Compose([
        transforms.ToTensor()
    ])

    # transform the image
    image = transform(image)
    # add a batch dimension
    image = image.unsqueeze(0).to(device)
    masks, boxes, labels = get_outputs(image, model, score)
    result = draw_segmentation_map(orig_image, masks, boxes, labels)
#     result = np.array(result)
    return result

In [ ]:
path = r'D:/maskRCNN/train_data/test'
count=1
for img in os.listdir(path):    
    
#     if (count >150): 
    print(img)
    result = single_img_predict(path, img, 0.5, model)
    dest = os.path.join(r'D:/maskRCNN/outputs', img)
    cv2.imwrite(dest, result)
    count = count +1
    if(count > 10):    
        break

In [ ]:
# visualize the image
result = cv2.cvtColor(np.array(result),cv2.COLOR_RGB2BGR)
cv2.imshow('Segmented image', result)
cv2.waitKey(0)